## Connect

In [3]:
from smsdk import client
from datetime import datetime, timedelta
import pandas as pd
import ipywidgets as widgets
import plotly.express as px

In [4]:
## Client 초기화 및 DB 연결

tenant = 'hansol-dj'
cli = client.Client(tenant)

In [5]:
## 인증 (apikey 방법)

key = 'cdf3b5ab-54d4-4fde-bc25-37667e2f4a31'
secret = 'sma_G38WRTxtc5AG7L1Rl4ndz2GNAup8Cx3i2KHGNepwZ18_'

cli.login('apikey', secret_id=secret, key_id=key)

True

## 데이터 검색
- 첫 번째 단계는 기계 유형을 조회한 다음 해당 유형의 특정 기계를 조회
 -> 한솔은 기계유형당 기계 1개임 : get_machine_type_names() = get_machine_names()

### Cycle 모델
- Cycle은 SM 플랫폼의 핵심 데이터 세트
- Cycle : 기계의 작업 단위를 나타내며 센서, 품질 관리 시스템, ERP, MES 등

In [32]:
# types = cli.get_machine_type_names()
# machine_type = types[0]

## machine(asset) 목록
machines = cli.get_machine_names()

## asset 선택 및 schema 확인

column_name = cli.get_machine_schema(machines[0])
# columns = cli.get_machine_schema(machines[0])['display'].to_list()

column_name['name'] = column_name['name'].str.replace('.', '__',regex=True)
column_name

,name,display,type
0,stats__DJ__PM31DCS__1PIC2420_M__val,6군 DRY 압력,float
1,stats__DJ__PM31DCS__1SI2807__MV__val,AFTER HOOD EXHAUST RPM,float
2,stats__DJ__PM31DCS__MIC2703_M__val,AFTER HOOD 수분량,float
3,stats__DJ__PM31DCS__PDIC2421_M__val,6군 DRY 차압,float
4,stats__DJ__PM31DCS__PDIC2421_O__val,6군 DRY 차압 V/V,float
5,stats__DJ__PM31DCS__PDIC2421_S__val,6군 DRY 차압 SET,float
6,stats__DJ__PM31DCS__PDIC2423_M__val,7군 TOP 차압,float
7,stats__DJ__PM31DCS__PDIC2423_O__val,7군 TOP 차압 V/V,float
8,stats__DJ__PM31DCS__PDIC2423_S__val,7군 TOP 차압 SET,float
9,stats__DJ__PM31DCS__PDIC2425_M__val,7군 BOTTOM 차압,float


In [39]:
## Cycle 데이터 검색

# row 생략 없이 출력
pd.set_option('display.max_rows', None)
# col 생략 없이 출력
pd.set_option('display.max_columns', None)

# select_columns = ['stats__DJ.PM31DCS.PDIC2421_S__val']

query = {'Machine': machines[0],
         'End Time__gte' : datetime(2022, 9, 1), 
         'End Time__lte' : datetime(2022, 9, 2), 
         '_order_by': '-End Time'
        #  '_only': select_columns,
        #  'stats__DJ.PM31DCS.PDIC2421_S__val__exists' : False
        }

df = cli.get_cycles(**query)

## 컬럼명 수정
display_dict = dict(zip(column_name.name, column_name.display))
df.rename(columns=display_dict, inplace=True)

## size 출력
print(f'Size of returned data: {df.shape}')

df.head(10)

_only not specified.  Selecting first 50 fields.
Size of returned data: (2881, 29)


,6군 DRY 압력,AFTER HOOD EXHAUST RPM,AFTER HOOD 수분량,6군 DRY 차압,6군 DRY 차압 V/V,6군 DRY 차압 SET,7군 TOP 차압,7군 TOP 차압 V/V,7군 TOP 차압 SET,7군 BOTTOM 차압,7군 BOTTOM 차압 V/V,7군 BOTTOM 차압 V/V SET,6군 DRY BY-PASS V/V,7군 TOP 압력,7군 TOP 압력 V/V,7군 TOP 압력 V/V SET,7군 BOTTOM 압력,7군 BOTTOM 압력 V/V,7군 BOTTOM 압력 V/V SET,AFTER HOOD ZERO LEVEL,AFTER HOOD 열풍 SUPPLY V/V,AFTER HOOD 열풍 SUPPLY V/V SET,Machine,Start Time,End Time,Cycle Time (Net),Cycle Time (Gross),Shift,Output
_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
rBSXJ5SmEDHWY7CVOkXp6Z9s5DsfO6ONYQfhOXklWyY=,0.004021,433.319335,19.119555,0.001762,100.0,NaN,0.086436,100.0,NaN,0.000793,100.0,NaN,0.0,0.0,100.0,NaN,0.122179,100.0,NaN,3.498670,30.0,NaN,After Dryer,2022-09-01 23:59:30,2022-09-02 00:00:00,30000.0,30000.0,CREW D,0.0
S85+RoknqzlfrgVEC5Vz00DpH8ai/eD2XkAB+7oXjbo=,0.004021,433.319335,19.119555,0.001762,100.0,NaN,0.086436,100.0,NaN,0.000793,100.0,NaN,0.0,0.0,100.0,NaN,0.123232,100.0,NaN,3.498670,30.0,NaN,After Dryer,2022-09-01 23:59:00,2022-09-01 23:59:30,30000.0,30000.0,CREW D,0.0
kBSbR9vxvhKT+l7nFgCWQ16g+E0q2UepEUviGhy1hq0=,0.004021,433.319335,19.119555,0.001762,100.0,NaN,0.086436,100.0,NaN,0.000793,100.0,NaN,0.0,0.0,100.0,NaN,0.123232,100.0,NaN,3.498670,30.0,NaN,After Dryer,2022-09-01 23:58:30,2022-09-01 23:59:00,30000.0,30000.0,CREW D,0.0
QxlgPjQYZhItouXeU4WO2ksvBkKmQiqziG6eRbKLOog=,0.004021,433.319335,19.119555,0.001762,100.0,NaN,0.086436,100.0,NaN,0.000793,100.0,NaN,0.0,0.0,100.0,NaN,0.123232,100.0,NaN,3.498670,30.0,NaN,After Dryer,2022-09-01 23:58:00,2022-09-01 23:58:30,30000.0,30000.0,CREW D,0.0
eEzO6qXGP/Ad+T3s4UxUh0Tnkyc2iBygbS1CnBs1ycY=,0.004021,433.319335,19.119555,0.001762,100.0,NaN,0.086436,100.0,NaN,0.000793,100.0,NaN,0.0,0.0,100.0,NaN,0.120974,100.0,NaN,3.498670,30.0,NaN,After Dryer,2022-09-01 23:57:30,2022-09-01 23:58:00,30000.0,30000.0,CREW D,0.0
F/drOfD0foL55/LigGgyRSO//Th2i3fCetRFUSlPemU=,0.004021,433.319335,19.119555,0.001762,100.0,NaN,0.086436,100.0,NaN,0.000793,100.0,NaN,0.0,0.0,100.0,NaN,0.120974,100.0,NaN,3.498670,30.0,NaN,After Dryer,2022-09-01 23:57:00,2022-09-01 23:57:30,30000.0,30000.0,CREW D,0.0
3xKm0vHxVFZFD1FjAx6txxPJXjeoT9yUeCrblWr51sM=,0.004021,433.319335,19.147699,0.001762,100.0,NaN,0.086436,100.0,NaN,0.000793,100.0,NaN,0.0,0.0,100.0,NaN,0.120974,100.0,NaN,3.498670,30.0,NaN,After Dryer,2022-09-01 23:56:30,2022-09-01 23:57:00,30000.0,30000.0,CREW D,0.0
RgKp1lRjKIDex+8E7BK2qgYd8z75jXyQq/Sgfa76NvI=,0.004021,433.319335,19.147699,0.001762,100.0,NaN,0.086436,100.0,NaN,0.000793,100.0,NaN,0.0,0.0,100.0,NaN,0.120974,100.0,NaN,3.498670,30.0,NaN,After Dryer,2022-09-01 23:56:00,2022-09-01 23:56:30,30000.0,30000.0,CREW D,0.0
zpGcd46usTa89Rfvo9aFc59Dmd5qa1Xt5xxlqaRtSBg=,0.004021,433.319335,19.160198,0.001762,100.0,NaN,0.086436,100.0,NaN,0.000793,100.0,NaN,0.0,0.0,100.0,NaN,0.119844,100.0,NaN,3.498670,30.0,3.498670,After Dryer,2022-09-01 23:55:30,2022-09-01 23:56:00,30000.0,30000.0,CREW D,0.0


### Parts 모델

In [36]:
part_types = cli.get_part_type_names()
part_type = part_types[0]

columns = cli.get_part_schema(part_type)['display'].to_list()

print(columns[:10])

['CAUSE_CODE', 'DJ.PM31DCS.11D_SP.SPD', 'DJ.PM31DCS.1CIC1122_M', 'DJ.PM31DCS.1FI1287_M', 'DJ.PM31DCS.1FIC1238_MV', 'DJ.PM31DCS.1FIQ2201_MV', 'DJ.PM31DCS.1FIQ2203_1_MV', 'DJ.PM31DCS.1FRIC1288.RATIO', 'DJ.PM31DCS.1FRIC4243.M', 'DJ.PM31DCS.1MIC2701_2']


In [37]:
query = {'Part': part_type,
         'End Time__gt': datetime(2022, 9, 1),
         'End Time__lt': datetime(2022, 9, 2),
         '#1 Broke Supply Pump Load__exists': True,
         '_limit': 10,
         '_only': columns[:30]}

df = cli.get_parts(**query)

print(f'Size of returned data: {df.shape}')
df.head()

None
Size of returned data: (0, 0)


Traceback (most recent call last):
  File "c:\Users\limjh\AppData\Local\Programs\Python\Python39\lib\site-packages\smsdk\ma_session.py", line 185, in _get_records_v1
    raise ValueError("Error - {}".format(response.text))
ValueError: Error - {"description":"Some of the submitted information was invalid","details":{"fields":{"where":{"list_items":{"3":{"fields":{"name":{"message":"Value must be one of: _id; type__part_type; serial; starttime; endtime; production_date_start; production_date_end; metadata__nettime; state; stats__CAUSE_CODE__val; stats__DJ.PM31DCS.11D_SP.SPD__val; stats__DJ.PM31DCS.1CIC1122_M__val; stats__DJ.PM31DCS.1FI1287_M__val; stats__DJ.PM31DCS.1FIC1238_MV__val; stats__DJ.PM31DCS.1FIQ2201_MV__val; stats__DJ.PM31DCS.1FIQ2203_1_MV__val; stats__DJ.PM31DCS.1FRIC1288.RATIO__val; stats__DJ.PM31DCS.1FRIC4243.M__val; stats__DJ.PM31DCS.1MIC2701_2__val; stats__DJ.PM31DCS.1MIC2701_2_OUT__val; stats__DJ.PM31DCS.1MIC2702_M2__val; stats__DJ.PM31DCS.1PDIC2411C_MV__val; stats__DJ.PM

""


## Timezones

In [27]:
cli.get_machine_timezone(machines[0])

'Asia/Seoul'

## 그래프 출력 기능?
- machine type, machine, predictor, outcome 선택
- 결과(outcome) 대 예측(predictor) 변수가 있는 산점도 출력


In [44]:
machine_types = cli.get_machine_type_names()

def dropdown_machine_types_event(change):
    machine_type = change.new
    machines = cli.get_machine_names(source_type=machine_type)
    
    dropdown_machines.options = machines
    dropdown_machines.value = None
    dropdown_machines.disabled = False

    dropdown_outcome.options = []
    dropdown_outcome.value = None
    dropdown_outcome.disabled = True
    
    dropdown_predictor.options = []
    dropdown_predictor.value = None
    dropdown_predictor.disabled = True
    
def dropdown_machines_event(change):
    machine = change.new
    fields = cli.get_machine_schema(machine, types=['int', 'float'])['display'].tolist()
    
    dropdown_outcome.options = fields
    dropdown_outcome.value = None
    dropdown_outcome.disabled = False
    
    dropdown_predictor.options = fields
    dropdown_predictor.value = None
    dropdown_predictor.disabled = False
    
def dropdown_outcome_event(change):
    run_analysis(dropdown_machines.value, change.new, dropdown_predictor.value)

def dropdown_predictor_event(change):
    run_analysis(dropdown_machines.value, dropdown_outcome.value, change.new)
    
def run_analysis(machine, outcome, predictor):
    output_df.clear_output()
    
    if machine and outcome and predictor:
    
        cols = ['Machine', 'End Time', outcome, predictor]

        query = {'Machine': machine,
                 'End Time__gt': datetime.utcnow() - timedelta(7),
                 '_limit': 500, 
                 '_only': cols
        }
        
        df = cli.get_cycles(**query)
        
        with output_df:
            if len(df):
                df.set_index('End Time', inplace=True)
                
                fig = px.scatter(df,
                         x = predictor,
                         y = outcome,
                         trendline = 'ols')
                
                display(fig)
                display(df[[outcome, predictor]])
            else:
                display(pd.DataFrame())
                
                
                
dropdown_machine_types = widgets.Dropdown(options = machine_types,
                                          value = None,
                                          description = 'Type',
                                          disabled = False)
dropdown_machines = widgets.Dropdown(options = [],
                                     value = None,
                                     description = 'Machine',
                                     disabled = True)
dropdown_outcome = widgets.Dropdown(options = [],
                                    value = None,
                                    description = 'Outcome',
                                    disabled = True)
dropdown_predictor = widgets.Dropdown(options = [],
                                      value = None,
                                      description = 'Predictor',
                                      disabled = True)
output_df = widgets.Output()

dropdown_machine_types.observe(dropdown_machine_types_event, names='value')
dropdown_machines.observe(dropdown_machines_event, names='value')
dropdown_outcome.observe(dropdown_outcome_event, names='value')
dropdown_predictor.observe(dropdown_predictor_event, names='value')

display(dropdown_machine_types)
display(dropdown_machines)
display(dropdown_outcome)
display(dropdown_predictor)
display(output_df)

Dropdown(description='Type', options=('After Dryer', 'BCTMP Line', 'Coater Dryer', 'Color Kitchen', 'District …

Dropdown(description='Machine', disabled=True, options=(), value=None)

Dropdown(description='Outcome', disabled=True, options=(), value=None)

Dropdown(description='Predictor', disabled=True, options=(), value=None)

Output()

None


Traceback (most recent call last):
  File "c:\Users\limjh\AppData\Local\Programs\Python\Python39\lib\site-packages\smsdk\ma_session.py", line 185, in _get_records_v1
    raise ValueError("Error - {}".format(response.text))
ValueError: Error - {"description":"An unexpected error occurred.","details":{},"error":"server_error"}



Dropping invalid column names: MACHIN SPEED.
